In [3]:
import timm

for model in timm.list_models():
    print(model)

/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_384
beit_large_patch16_512
beitv2_base_patch16_224
beitv2_large_patch16_224
botnet26t_256
botnet50ts_256
caformer_b36
caformer_m36
caformer_s18
caformer_s36
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_medium
coat_lite_medium_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_small
coat_tiny
coatnet_0_224
coatnet_0_rw_224
coatnet_1_224
coatnet_1_rw_224
coatnet_2_224
coatnet_2_rw_224
coatnet_3_224
coatnet_3_rw_224
coatnet_4_224
coatnet_5_224
coatnet_bn_0_rw_224
coatnet_nano_cc_224
coatnet_nano_rw_224
coatnet_pico_rw_224
coatnet_rmlp_0_rw_224
coatnet_rmlp_1_rw2_224
coatnet_rmlp_1_rw_224
coatnet_rmlp_2_rw_224
coatnet_rmlp_2_rw_384
coatnet_rmlp_3_rw_224
coatnet_rmlp_nano_rw_224
coatnext_nano_rw_224
convformer_b36
convformer_m36
convformer_s18
convformer_s36
convit_base
conv

In [1]:
import timm
model = timm.create_model('nextvit_base', pretrained=True)
model = model

/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import torch.nn as nn

# 기존 모델의 head 부분을 제외하고 새로운 모델을 정의합니다.
model_without_head = nn.Sequential(*list(model.children())[:-1])
model_without_head

Sequential(
  (0): Sequential(
    (0): ConvNormAct(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (1): ConvNormAct(
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (2): ConvNormAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (3): ConvNormAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplac

In [3]:
nn.Sequential(*list(model.children())[:1])

Sequential(
  (0): Sequential(
    (0): ConvNormAct(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (1): ConvNormAct(
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (2): ConvNormAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (3): ConvNormAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplac

In [2]:
import torch.nn as nn

# 기존 모델에서 head 부분을 제외
model_without_head = nn.Sequential(*list(model.children())[:-1])

# 새로운 detection head 정의 (예시로 Conv2d와 Linear 사용)
class DetectionHead(nn.Module):
    def __init__(self, in_features, num_classes, num_boxes):
        super(DetectionHead, self).__init__()
        # Detection-specific layers 예시 (Conv2d 또는 FC 레이어 등)
        self.conv = nn.Conv2d(in_features, 256, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)  # 적절한 입력 크기로 설정
        self.fc2 = nn.Linear(1024, num_classes * num_boxes * 4)  # Bounding box 예측 (좌표, 클래스)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)  # Flattening
        x = self.fc1(x)
        x = self.fc2(x)
        return x.view(x.size(0), -1, 4)  # 예시로 [batch, num_boxes, 4] 형태

# 새로운 detection head 추가
in_features = 1024  # 기존 모델의 마지막 레이어 출력 크기 확인 후 설정
num_classes = 20    # 예시로 20개 클래스
num_boxes = 5       # 예시로 5개 박스 예측

detection_head = DetectionHead(in_features, num_classes, num_boxes)

# 전체 모델
class ModelWithDetectionHead(nn.Module):
    def __init__(self, backbone, head):
        super(ModelWithDetectionHead, self).__init__()
        self.backbone = backbone
        self.head = head

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

# 모델 초기화
model_with_detection_head = ModelWithDetectionHead(model_without_head, detection_head)
